<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240115-%E8%BF%87%E6%8B%9F%E5%90%88%E6%A3%80%E6%9F%A5NDRE3class_HANTS%E4%BA%8C%E9%98%B6RMSE%E3%80%81MAE%E3%80%81R2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [12]:
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [13]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [14]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2020-01-01','2023-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B5']).rename('NDRE')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

1989


In [15]:
dataset = filteredSentinel.select(['NDRE','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDRE')

In [16]:
# 二阶谐波
harmonicIndependents_2 = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2'])

def add_sincos_2(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2'))

harmonicSentinel_2 = dataset.map(add_sincos_2)

harmonicTrend_2 = harmonicSentinel_2.select(harmonicIndependents_2.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_2.length(),numY=1))

harmonicTrendCoefficients_2 = harmonicTrend_2.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_2])

def fit_2(image):
  return image.addBands(image.select(harmonicIndependents_2).multiply(harmonicTrendCoefficients_2).reduce('sum').rename('fitted'))

fittedHarmonic_2 = harmonicSentinel_2.map(fit_2)

In [17]:
composite1 = s2sr.filterBounds(roi).filterDate('2020-01-01','2024-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)
filteredSentinel1 = composite1.map(s2addVariables)
dataset1 = filteredSentinel1.select(['NDRE','t','constant']);
harmonicSentinel1 = dataset1.map(add_sincos_2)
fittedHarmonic1 = harmonicSentinel1.map(fit_2)

# 评价拟合效果

In [18]:
points = [ee.Geometry.Point(103.69442853431933, 30.551562081391808),ee.Geometry.Point(103.65488522617737, 30.533297603476893),ee.Geometry.Point(103.59582889182555, 30.52539161730913),ee.Geometry.Point(103.56952465471126, 30.499742868049715),ee.Geometry.Point(103.53194781013387, 30.53031446487214),ee.Geometry.Point(103.50069397085026, 30.55269915216044),ee.Geometry.Point(103.46811131945273, 30.532325519447248),ee.Geometry.Point(103.49599853744098, 30.5045720744906),ee.Geometry.Point(103.53704010353692, 30.476564460773904),ee.Geometry.Point(103.56092304877916, 30.447647190442307),ee.Geometry.Point(103.59226607008384, 30.446978325284288),ee.Geometry.Point(103.62079985772485, 30.439781336184026),ee.Geometry.Point(103.59236417030701, 30.470914779739182),ee.Geometry.Point(103.68473444862916, 30.49993015029396),ee.Geometry.Point(103.75471558056192, 30.43588854096437),ee.Geometry.Point(103.84447728476779, 30.443179171186756),ee.Geometry.Point(103.85018047501515, 30.467788949900722),ee.Geometry.Point(103.6853230499682, 30.687827750484054),ee.Geometry.Point(103.65355195496241, 30.710867925627216),ee.Geometry.Point(103.6298830465707, 30.742799548270945),ee.Geometry.Point(103.59532947250756, 30.768805025614643),ee.Geometry.Point(103.56548916825788, 30.793976651061346),ee.Geometry.Point(103.72427329767005, 30.713917950747774),ee.Geometry.Point(103.74967233727047, 30.683810100434908),ee.Geometry.Point(103.71968488268605, 30.600692456798672),ee.Geometry.Point(103.74770587370591, 30.576323469541602),ee.Geometry.Point(103.74953856423888, 30.553011289234185),ee.Geometry.Point(103.7740948337402, 30.5171868713708),ee.Geometry.Point(103.81560460544674, 30.52750077210742),ee.Geometry.Point(103.81192584707763, 30.56217028279795),ee.Geometry.Point(103.81033394800156, 30.575315712703514),ee.Geometry.Point(103.77831760243781, 30.573233312511555),ee.Geometry.Point(103.91586749263347, 30.848234992679675),ee.Geometry.Point(103.85799727916184, 30.84885926682715),ee.Geometry.Point(103.85485361291917, 30.80372870506523),ee.Geometry.Point(103.79825870234883, 30.829515686457306),ee.Geometry.Point(103.74891874919244, 30.8550618763925),ee.Geometry.Point(103.79212743840034, 30.902011751384222),ee.Geometry.Point(103.82710462706356, 30.885321336141182),ee.Geometry.Point(103.85819793870925, 30.90270291204751),ee.Geometry.Point(103.82015288852125, 30.935334613556623),ee.Geometry.Point(103.75618708390955, 30.985240872546797),ee.Geometry.Point(103.75686040816862, 30.939191735967864),ee.Geometry.Point(103.7038372375421, 30.93940131371737),ee.Geometry.Point(103.88234843001472, 31.089717610229034),ee.Geometry.Point(103.95180338802317, 31.09406077465508),ee.Geometry.Point(103.95662367626194, 31.122732794428746),ee.Geometry.Point(104.00554001481827, 31.09055592122708),ee.Geometry.Point(104.03197802496413, 31.058004483536926),ee.Geometry.Point(103.98008746600517, 31.060274165973127),ee.Geometry.Point(103.9804218985842, 31.010140492829244),ee.Geometry.Point(104.03809591160946, 31.00245746171415),ee.Geometry.Point(104.07186914298828, 31.02673280784911),ee.Geometry.Point(104.10680619974202, 31.003808569333344),ee.Geometry.Point(104.10340390563823, 30.972006260620166),ee.Geometry.Point(104.1242011529515, 30.924481161592546),ee.Geometry.Point(104.07048682166169, 30.936092660735714),ee.Geometry.Point(104.04891369076513, 30.929791950947198),ee.Geometry.Point(104.03893868170857, 30.95408959258743),ee.Geometry.Point(104.03782836554626, 30.901454363752542),ee.Geometry.Point(104.07251125353997, 30.899523572996404),ee.Geometry.Point(104.12622558482975, 30.876773239421727),ee.Geometry.Point(104.16596955251913, 30.86185754639795),ee.Geometry.Point(104.19711637337745, 30.843508345563016),ee.Geometry.Point(104.20473697707779, 30.787947946437082),ee.Geometry.Point(104.2233002147633, 30.74701339876645),ee.Geometry.Point(104.32238144017084, 30.867235222268405),ee.Geometry.Point(104.34800343482397, 30.86287868053919),ee.Geometry.Point(104.32160109748651, 30.792647838947417),ee.Geometry.Point(103.68982228293112, 30.47573060887691),ee.Geometry.Point(103.682803657873, 30.42125377130696),ee.Geometry.Point(103.65444377517308, 30.390432464825544),ee.Geometry.Point(103.62306954016104, 30.394566051502085),ee.Geometry.Point(103.43668803432911, 30.473443090036486),ee.Geometry.Point(103.56000447396217, 30.390294232692884),ee.Geometry.Point(103.67927204983867, 30.311903236173308),ee.Geometry.Point(103.76919428168246, 30.41811010506428),ee.Geometry.Point(103.68609447445044, 30.57786631850608),ee.Geometry.Point(103.58707121735658, 30.56275888413701),ee.Geometry.Point(103.62182991006817, 30.5831235986481),ee.Geometry.Point(103.61745107283369, 30.616794270702663),ee.Geometry.Point(103.56919022213226, 30.62262677488058),ee.Geometry.Point(103.59710419472684, 30.66326256278062),ee.Geometry.Point(103.63035125218131, 30.69151988615631),ee.Geometry.Point(103.62576729629838, 30.63486254817123),ee.Geometry.Point(103.6501897927681, 30.606542797380797),ee.Geometry.Point(103.6639639559222, 30.67538685854495),ee.Geometry.Point(103.81523004095824, 30.638768720694053),ee.Geometry.Point(103.87737653234012, 30.7408687575148),ee.Geometry.Point(103.90792137455621, 30.738166542276414),ee.Geometry.Point(103.81904703145999, 30.766843021151118),ee.Geometry.Point(103.63658061635299, 30.784545652333307),ee.Geometry.Point(103.62725217694917, 30.828244842657078),ee.Geometry.Point(103.62660560729641, 30.88052334340767),ee.Geometry.Point(103.632411356868, 30.939231867877343),ee.Geometry.Point(103.67458107553044, 31.064180338495948),ee.Geometry.Point(103.70539346380976, 31.035976524332895),ee.Geometry.Point(104.01145724191619, 30.97983644207001),ee.Geometry.Point(103.98222783451084, 30.95264930294715),ee.Geometry.Point(103.9124830349339, 30.953741782705258),ee.Geometry.Point(103.85152712353329, 30.95874043498617),ee.Geometry.Point(103.79579727856733, 30.96167452347934),ee.Geometry.Point(103.78737403667736, 31.040792353470632),ee.Geometry.Point(103.85454147584541, 31.06187052415026),ee.Geometry.Point(103.88376196504863, 31.045844514964173),ee.Geometry.Point(103.84999319277091, 31.0103456114777),ee.Geometry.Point(103.91530118679968, 31.007634478037204),ee.Geometry.Point(103.97618575258342, 30.898966185364717),ee.Geometry.Point(103.94299666344261, 30.873669705088535),ee.Geometry.Point(103.97305992274497, 30.848172565264928),ee.Geometry.Point(103.99353165568132, 30.86875577572763),ee.Geometry.Point(104.04493617262546, 30.849260585921964),ee.Geometry.Point(104.10052778545875, 30.837296817795565),ee.Geometry.Point(104.12120909614465, 30.817378013389817),ee.Geometry.Point(103.84901664964019, 31.010283184062946),ee.Geometry.Point(103.71719224519717, 30.472814356787953),ee.Geometry.Point(103.68974647821322, 30.475694936068486),ee.Geometry.Point(103.6548450942679, 30.472595860836336),ee.Geometry.Point(103.65476483044894, 30.472903538809025),ee.Geometry.Point(103.651349159042, 30.446982784385344),ee.Geometry.Point(103.5293481542197, 30.420950552435325),ee.Geometry.Point(103.4988523621152, 30.450113073324868),ee.Geometry.Point(103.4963240518179, 30.47491905248518),ee.Geometry.Point(103.49697062147064, 30.474923511586237),ee.Geometry.Point(103.44297090771342, 30.503831863715725),ee.Geometry.Point(103.436179696809, 30.47350105835019),ee.Geometry.Point(103.43450307481291, 30.42161049939123),ee.Geometry.Point(103.46724625384837, 30.446911438768485),ee.Geometry.Point(103.61455711625003, 30.36449387399765),ee.Geometry.Point(103.68620595197677, 30.358759470042916),ee.Geometry.Point(103.71908290404384, 30.3600927412579),ee.Geometry.Point(103.77645815729852, 30.389710090454876),ee.Geometry.Point(103.77456303935081, 30.390454760330815),ee.Geometry.Point(103.8763241844915, 30.416678733626124),ee.Geometry.Point(103.87105798614739, 30.445970568446214),ee.Geometry.Point(103.68606326074307, 30.577705790868162),ee.Geometry.Point(103.65106823567564, 30.55970885901645),ee.Geometry.Point(103.62362246869168, 30.550754984101136),ee.Geometry.Point(103.59622129271824, 30.715059480617448),ee.Geometry.Point(103.59917321761561, 30.713730668503533),ee.Geometry.Point(103.63093985352037, 30.691537722560533),ee.Geometry.Point(103.90764936939195, 30.73843408833962),ee.Geometry.Point(103.84272485805378, 30.733974987286174),ee.Geometry.Point(103.81973373302223, 30.766521965875278),ee.Geometry.Point(103.70528198628341, 31.036101379162403),ee.Geometry.Point(103.70293204002824, 30.93987397842904),ee.Geometry.Point(103.7489321264956, 30.855146599312512),ee.Geometry.Point(103.82210597478266, 30.875426590903576),ee.Geometry.Point(103.85075569905104, 30.906560034458742),ee.Geometry.Point(103.91618408880825, 30.901927028464208),ee.Geometry.Point(103.98263807180777, 30.952635925644),ee.Geometry.Point(103.85198195184073, 30.958967849139892),ee.Geometry.Point(103.75965626452916, 31.013016613008702),ee.Geometry.Point(103.78707081780573, 31.040948422007496),ee.Geometry.Point(103.85476888999914, 31.061754587522877),ee.Geometry.Point(103.88393586998974, 31.045719660134687),ee.Geometry.Point(103.84999765187194, 31.010109279121863),ee.Geometry.Point(103.91554197825657, 31.007839596685663),ee.Geometry.Point(103.94347824635639, 30.87328176329688),ee.Geometry.Point(103.99313925478862, 30.868800366738167),ee.Geometry.Point(104.04520817778972, 30.849577182096763),ee.Geometry.Point(104.12120909614465, 30.81684292126341),ee.Geometry.Point(103.86281310829955, 30.532307683043033),ee.Geometry.Point(103.88304404977903, 30.514056582431277),ee.Geometry.Point(104.32233239005927, 30.893909564770112),ee.Geometry.Point(103.7171610314898, 30.54972493175779),ee.Geometry.Point(103.4787774891726, 30.511684340670843),ee.Geometry.Point(103.74441505712846, 30.49009337337006),ee.Geometry.Point(103.73493054918778, 30.66060493855278),ee.Geometry.Point(103.53428437908589, 30.77065555255182),ee.Geometry.Point(103.69229262491474, 30.769362413246316),ee.Geometry.Point(103.69146769121986, 30.7438385188164),ee.Geometry.Point(103.78781548768166, 30.60887490723174),ee.Geometry.Point(103.99598862036179, 30.746968807755923),ee.Geometry.Point(104.0033996463126, 30.766624525199504),ee.Geometry.Point(103.91834675281916, 30.896687584726415),ee.Geometry.Point(103.94983246535755, 31.043904806005933),ee.Geometry.Point(103.71711644047926, 30.472925834314296),ee.Geometry.Point(103.65482279876264, 30.472685042857403),ee.Geometry.Point(103.49845550212143, 30.450358323882796),ee.Geometry.Point(103.49697062147064, 30.474923511586237),ee.Geometry.Point(103.43403486920232, 30.422065327698693),ee.Geometry.Point(103.46697870778517, 30.44713439382116),ee.Geometry.Point(103.68610339265254, 30.35813965499649),ee.Geometry.Point(103.71580992387062, 30.359718176769412),ee.Geometry.Point(103.77456303935081, 30.390454760330815),ee.Geometry.Point(103.83757459633706, 30.3949673705969),ee.Geometry.Point(103.87085286749893, 30.44555587204824),ee.Geometry.Point(103.71971163729236, 30.600317892310187),ee.Geometry.Point(103.653436018335, 30.561893818532642),ee.Geometry.Point(103.84408934297613, 30.73422915604622),ee.Geometry.Point(103.59517340397068, 30.85430382921341),ee.Geometry.Point(103.60227229284777, 30.991452400314245),ee.Geometry.Point(103.76001745171446, 31.012842708067623),ee.Geometry.Point(103.84697884045877, 30.60314050327702),ee.Geometry.Point(103.85057733500891, 30.543642717920893),ee.Geometry.Point(103.58007488780372, 30.39918568019346),ee.Geometry.Point(103.38049444285357, 30.3478703452704),ee.Geometry.Point(103.64891894896788, 30.346724356299667),ee.Geometry.Point(103.63296874449968, 30.3443744100445),ee.Geometry.Point(103.65282958059173, 30.42232395555979),ee.Geometry.Point(103.64037085224842, 30.423376303408403),ee.Geometry.Point(103.63730299072364, 30.419470130885575),ee.Geometry.Point(103.57229821556652, 30.486588519942053),ee.Geometry.Point(103.57890660332772, 30.485237412322856),ee.Geometry.Point(103.56894943067537, 30.4896385450626),ee.Geometry.Point(103.50341402249288, 30.536026573321603),ee.Geometry.Point(103.50989755542459, 30.54068633392245),ee.Geometry.Point(103.50667362536294, 30.53915686226112),ee.Geometry.Point(103.76578752847765, 30.6029799756391),ee.Geometry.Point(103.76479760804376, 30.599278921764732),ee.Geometry.Point(103.75789046051199, 30.596304701362087),ee.Geometry.Point(103.6881010699245, 30.71963897739935),ee.Geometry.Point(103.68843550250351, 30.71451992938999),ee.Geometry.Point(103.75723497265712, 30.575364762815095),ee.Geometry.Point(103.76006650182606, 30.57438376058334),ee.Geometry.Point(103.76693351744836, 30.573112916783117),ee.Geometry.Point(103.75418494753656, 30.570397324241565),ee.Geometry.Point(103.82964631466402, 30.44247463322031),ee.Geometry.Point(103.85137105499642, 30.4545721743783),ee.Geometry.Point(103.86083326743183, 30.47900804815119),ee.Geometry.Point(103.86151996899407, 30.482325619334954),ee.Geometry.Point(103.84016979315015, 30.570062891662555),ee.Geometry.Point(103.85125957747009, 30.53796628227985),ee.Geometry.Point(103.8680971430479, 30.513302994353243),ee.Geometry.Point(103.88924220024334, 30.4387022337291),ee.Geometry.Point(103.77969546466332, 30.38411837773386),ee.Geometry.Point(103.76709850418735, 30.385340171422502),ee.Geometry.Point(103.77269913511049, 30.38256215146621),ee.Geometry.Point(103.76153354607264, 30.81045302945383),ee.Geometry.Point(103.7014516184785, 30.947106640337726),ee.Geometry.Point(103.73196524698724, 30.947218117864068),ee.Geometry.Point(103.72080411705046, 31.011861705835866),ee.Geometry.Point(103.72374266464469, 31.015870437682914),ee.Geometry.Point(103.72972677825841, 31.00976592834075),ee.Geometry.Point(103.90301636339744, 30.968028742480495),ee.Geometry.Point(103.93019458431819, 30.959048112958854),ee.Geometry.Point(103.89900763155038, 30.864064801419403),ee.Geometry.Point(103.84756744179784, 30.805873532671935),ee.Geometry.Point(103.77078618075855, 30.707131198944428),ee.Geometry.Point(104.03428783930983, 30.293384589498338),ee.Geometry.Point(104.03901002732543, 30.28988419517139),ee.Geometry.Point(104.07633270314275, 31.00191345138563),ee.Geometry.Point(104.05235165767732, 30.97535058641025),ee.Geometry.Point(104.05664131289073, 30.97367842351521),ee.Geometry.Point(104.12643070347822, 30.87721914952707),ee.Geometry.Point(104.12990880229991, 30.875613873147838)]

In [19]:
result_df_train = pd.DataFrame(columns=['RMSE', 'MAE', 'R2'])
result_df_valid = pd.DataFrame(columns=['RMSE', 'MAE', 'R2'])

for point in points:
    rTable = ee.FeatureCollection(fittedHarmonic1.filterBounds(point).map(lambda image: ee.Feature(
        None,
        {
            'date': image.date().format('yyyy-MM-dd'),
            'NDRE': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDRE"),
            'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
        }
    )))

    rTable_dict = rTable.getInfo()

    dates = [feat['properties']['date'] for feat in rTable_dict['features']]
    NDRE_values = [feat['properties']['NDRE'] for feat in rTable_dict['features']]
    fitted_values = [feat['properties']['fitted'] for feat in rTable_dict['features']]

    df = pd.DataFrame({'Date': dates, 'NDRE': NDRE_values, 'Fitted': fitted_values})

    df = df.dropna()

    if not df.empty:
        df.isetitem(df.columns.get_loc('Date'), pd.to_datetime(df['Date']))

        train_df = df[df['Date'] < '2023-01-01']
        valid_df = df[df['Date'] >= '2023-01-01']

        y_train = train_df['NDRE']
        y_valid = valid_df['NDRE']

        y_train_pred = train_df['Fitted']
        y_valid_pred = valid_df['Fitted']

        rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
        mae_train = mean_absolute_error(y_train, y_train_pred)
        r2_train = r2_score(y_train, y_train_pred)

        rmse_valid = mean_squared_error(y_valid, y_valid_pred, squared=False)
        mae_valid = mean_absolute_error(y_valid, y_valid_pred)
        r2_valid = r2_score(y_valid, y_valid_pred)

        result_df_train = pd.concat([result_df_train, pd.DataFrame([{'RMSE': rmse_train, 'MAE': mae_train, 'R2': r2_train}])], ignore_index=True)
        result_df_valid = pd.concat([result_df_valid, pd.DataFrame([{'RMSE': rmse_valid, 'MAE': mae_valid, 'R2': r2_valid}])], ignore_index=True)

rmse_train = np.mean(result_df_train['RMSE'])
mae_train = np.mean(result_df_train['MAE'])
r2_train = np.mean(result_df_train['R2'])
print(rmse_train, mae_train, r2_train)

rmse_valid = np.mean(result_df_valid['RMSE'])
mae_valid = np.mean(result_df_valid['MAE'])
r2_valid = np.mean(result_df_valid['R2'])
print(rmse_valid, mae_valid, r2_valid)

0.09048288609484223 0.07260152113989644 0.5854725844856369
0.12487524197431214 0.10134328168370382 0.004391653700317748


In [20]:
result_df_train = result_df_train.dropna()
result_df_valid = result_df_valid.dropna()

In [21]:
rmse_train = np.median(result_df_train['RMSE'])
mae_train = np.median(result_df_train['MAE'])
r2_train = np.median(result_df_train['R2'])
print(rmse_train, mae_train, r2_train)

rmse_valid = np.median(result_df_valid['RMSE'])
mae_valid = np.median(result_df_valid['MAE'])
r2_valid = np.median(result_df_valid['R2'])
print(rmse_valid, mae_valid, r2_valid)

0.09040270532823878 0.07181323684298678 0.6107259524813189
0.11807793177455478 0.09374614595196896 0.2198086834458597
